## Classification of Income Being Above or Below/Equal to $50,000 in Adult Census Data

By: Sunsar, Sarah, Emily, Calvin (DSCI 100 003 - Group 23)

Data is from: https://www.kaggle.com/datasets/uciml/adult-census-income

The dataset used for this analysis is derived from the 1994 Census Bureau database. 

The dataset contains a diverse range of numerical and categorical attributes, such as age, hours worked per week, and race. In this project, we will filter and simplify some categories from this dataset to predict whether an individual's annual salary falls above or below $50,000.

The question this project will seek to answer is: 

**How do different aspects of a person predict whether annual income will be above or below $50K annually?**

# Preliminary Exploratory Data Analysis

In [79]:
library(tidyverse)
library(tidymodels)


In [84]:
df = read_csv('https://raw.githubusercontent.com/calvingdu/dsci100-003-23/master/data/adult_census.csv')

# Splitting the data
df_split <- initial_split(df, prop = 0.8, strata = income)
df_train <- training(df_split)
df_test <- testing(df_split)

paste0("Training set row count: ", nrow(df_train))
paste0("Testing set row count: ", nrow(df_test))

Rows: 32561 Columns: 15
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (9): workclass, education, marital.status, occupation, relationship, rac...
dbl (6): age, fnlwgt, education.num, capital.gain, capital.loss, hours.per.week

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "Training set row count: 26048"

[1] "Testing set row count: 6513"

In [85]:
head(df_train)

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K


# Tidying/Cleaning The Data

In [94]:
impute_scale_recipe <- recipe(income ~ ., data = df) %>%
  step_scale(all_numeric_predictors()) %>%
  step_center(all_numeric_predictors()) %>%
  step_impute_mean(all_numeric_predictors()) %>%
  prep()

tidy_df <- bake(impute_scale_recipe, df)

unique(tidy_df$workclass)
# can notice there are ? 
unique(tidy_df$occupation)
# can notice there are ?

unique(tidy_df$relationship)

[1] ?                Private          State-gov        Federal-gov     
[5] Self-emp-not-inc Self-emp-inc     Local-gov        Without-pay     
[9] Never-worked    
9 Levels: ? Federal-gov Local-gov Never-worked Private ... Without-pay

[1] ?                 Exec-managerial   Machine-op-inspct Prof-specialty   
 [5] Other-service     Adm-clerical      Craft-repair      Transport-moving 
 [9] Handlers-cleaners Sales             Farming-fishing   Tech-support     
[13] Protective-serv   Armed-Forces      Priv-house-serv  
15 Levels: ? Adm-clerical Armed-Forces Craft-repair ... Transport-moving

[1] Widowed               Divorced              Separated            
[4] Never-married         Married-civ-spouse    Married-spouse-absent
[7] Married-AF-spouse    
7 Levels: Divorced Married-AF-spouse ... Widowed

[1] Not-in-family  Unmarried      Own-child      Other-relative Husband       
[6] Wife          
Levels: Husband Not-in-family Other-relative Own-child Unmarried Wife

# Which Variables To Use: 
We noticed immediately that there are a lot of categorical columns. To tackle this, we plan to make buckets of categories and then turn them into dummy variables. For example, we can turn sex into a dummy variable where 1 = male and 0 = female, make dummy variables for different categories in working class (government, self-employed, private), etc. 

These are the columns we plan to use and how we plan to tackle them: 
- **Age**: No changes (either than scaling/imputation)
- **Workclass**: Make a dummy variable for in gov, self employed, private 
- **Education**: Simplified to College Graduate or Not
- **Occupation**: Make a dummy variable for all the occupations
- **Relationship**: Simplified to Married or Not
- **Sex**: Transformed into a Dummy Variable
- **Capital Gain**: No changes (either than scaling/imputation)
- **Capital Loss**: No changes (either than scaling/imputation)
- **Hours Per Week**: No changes (either than scaling/imputation)
- **Native Country**: Evaluate what country has the most people with income > 50k and then make a dummy variable for being in this country or not 

** Dropped Columns & Reasoning**: 
- fnlwgt: Unclear how this relates
- education.num: Already have education
- race: Don't believe this is significant to the study so removing to avoid overfitting 
- marital.status: We think it's similar to relationship so we remove to avoid overfitting 


In [86]:

most_rich_country <- df_train %>%
  filter(income == '>50K') %>%
  group_by(native.country) %>%
  summarize(count = n()) %>%
  arrange(desc(count)) %>%
  slice(1)

most_rich_country_name <- most_rich_country$native.country

transformed_data <- df_train %>%
  mutate(is_united_states = ifelse(native.country == most_rich_country_name, 1, 0))
transformed_data

age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,is_united_states
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K,1
82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K,1
66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K,1
54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K,1
34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K,1
38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K,1
68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40,United-States,<=50K,1
29,Private,77009,11th,7,Separated,Sales,Not-in-family,White,Female,0,2754,42,United-States,<=50K,1
61,Private,29059,HS-grad,9,Divorced,Sales,Unmarried,White,Female,0,2754,25,United-States,<=50K,1


age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,is_united_states
<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K,1
82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K,1
66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K,1
54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K,1
41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K,1
34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K,1
38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K,1
74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20,United-States,>50K,1
68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40,United-States,<=50K,1
